In [1]:
%mkdir -p /content/drive/MyDrive/GSE165291
%cd /content/drive/MyDrive/GSE165291

/content/drive/MyDrive/GSE165291


In [ ]:
!wget "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE165291&format=file" -O GSE165291_RAW.tar
!tar -xvf GSE165291_RAW.tar

--2024-08-13 07:19:59--  https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE165291&format=file
Resolving www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)... 130.14.29.110, 2607:f220:41e:4290::110
Connecting to www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)|130.14.29.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1258455040 (1.2G) [application/x-tar]
Saving to: ‘GSE165291_RAW.tar’

GSE165291_RAW.tar   100%[===================>]   1.17G  41.5MB/s    in 32s     

2024-08-13 07:20:32 (37.1 MB/s) - ‘GSE165291_RAW.tar’ saved [1258455040/1258455040]

GSM5029515_CRISPRn_screen_virus_processed.tar.gz
GSM5029516_CRISPRi_screen_host_processed.tar.gz
GSM5029517_CRISPRn_screen_host_processed.tar.gz
GSM5029518_unperturbed_time_course_processed.tar.gz
GSM5029519_CRISPRi_perturb_host_processed.tar.gz
GSM5029520_CRISPRn_perturb_virus_host_processed.tar.gz


In [ ]:
%%bash

for file in $(find . -maxdepth 1 -type f -name '*.tar.gz'); do
    mkdir -p $(basename file)
    tar -xvf $file -C $(basename file)
done

In [ ]:
%ls GSM5029519_CRISPRi_perturb_host_processed/processed
%ls GSM5029520_CRISPRn_perturb_virus_host_processed/processed

barcodes.tsv  cell_identities.csv  genes.tsv  matrix.mtx
barcodes.tsv  cell_identities.csv  genes.tsv  matrix.mtx


In [ ]:
!head GSM5029519_CRISPRi_perturb_host_processed/processed/matrix.mtx
!head GSM5029520_CRISPRn_perturb_virus_host_processed/processed/matrix.mtx

%%MatrixMarket matrix coordinate integer general
%
34057 46364 149895358
9 1 1
33 1 5
34 1 2
50 1 1
53 1 1
59 1 2
60 1 6
%%MatrixMarket matrix coordinate integer general
%
34057 55661 184919406
28 1 1
33 1 1
34 1 20
36 1 1
46 1 4
50 1 3
56 1 1


In [2]:
%pip install scanpy python-dotenv -q

import os
import tarfile
import json
import gzip

import scanpy as sc
import anndata as ad
import pandas as pd

from dotenv import load_dotenv
load_dotenv(dotenv_path='/content/drive/MyDrive/.gse.env')
COLS = json.loads(os.environ.get('OBS_COLS'))
ORGANISMS = json.loads(os.environ.get('ORGANISMS'))

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 5.3 MB/s eta 0:00:00


In [ ]:
from pathlib import Path
import numpy as np

def read_10x_mtx_legacy(path, prefix=''):
    path = Path(path)
    adata = sc.read_mtx(path / f"{prefix}matrix.mtx").T
    barcodes = pd.read_csv(path / f"{prefix}barcodes.tsv", header=None, sep='\t')
    barcodes.columns = ['cell_barcode']
    adata.obs.index = barcodes['cell_barcode']
    genes = pd.read_csv(path / f"{prefix}genes.tsv", header=None, sep='\t', index_col=1)
    genes.index.name = 'gene_name'
    genes.columns = ['gene_id']
    adata.var = genes
    return adata

In [ ]:
adatas = {}
for sample in ['GSM5029519_CRISPRi_perturb_host_processed/processed', 'GSM5029520_CRISPRn_perturb_virus_host_processed/processed']:
    adata = read_10x_mtx_legacy(sample)
    obs = pd.read_csv(f'{sample}/cell_identities.csv', index_col=0)
    indicies = np.intersect1d(adata.obs_names, obs.index)

    adata = adata[indicies]
    obs = obs.loc[indicies]
    adata.obs = obs
    adata.obs_names_make_unique()
    adata.var_names_make_unique()
    adatas[sample.split('_')[3]] = adata

adata = ad.concat(adatas, label='sample')
adata.write_h5ad("./GSE165291_merged.h5ad")

/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [ ]:
ORGANISMS

['Humans (Homo sapiens)',
 'Mice (Mus musculus)',
 'Rats (Rattus norvegicus)',
 'Zebrafish (Danio rerio)',
 'Fruit flies (Drosophila melanogaster)',
 'Nematodes (Caenorhabditis elegans)',
 'Yeast (Saccharomyces cerevisiae)',
 'Dogs (Canis lupus familiaris)',
 'Pigs (Sus scrofa)']

In [ ]:
adatas['host'].obs['guide_identity'].str.split('_').str[0].unique()

array(['TRAPPC11', 'STAT2', 'CASP9', 'HCCS', 'EXT2', 'LAMTOR4', 'SLC35B2',
       'C17orf59', 'KXD1', 'CUL3', 'BCL2L1', 'NEDD8', 'GLCE', 'CYCS',
       'SEC62', 'TRAPPC12', 'VTCN1', 'CSNK2A1', 'B4GALT7', 'AMFR',
       'SEC63', 'TRAF2', 'LAMTOR1', 'TRAPPC8', 'KIAA1432', 'GFP', 'BAG6',
       'COG5', 'RBX1', 'RGP1', 'RAB6A', 'LARGE', 'HS6ST1', 'LAMTOR2',
       'ASNA1', 'NAE1', 'RBBP5', 'DDA1', 'FAM126A', 'SEC61B', '*',
       'IFNAR2', 'UBA3', 'COG2', 'COG8', 'CASP3', 'LAMTOR3', 'CASP2',
       'WDR26', 'DCAF4', 'IRF9', 'WDR81', 'KIRREL', 'LAMTOR5', 'UNC50'],
      dtype=object)

In [ ]:
adata.obs['sample'] = adata.obs['sample'].apply(
    lambda x: x if x == 'host' else 'host_and_virus'
)

In [ ]:
adata.obs['cancer_type'] = 'non-cancer'
adata.obs['cell_type'] = 'human foreskin fibroblasts (HFFs)'
adata.obs['organism'] = ORGANISMS[0]
adata.obs.loc[adata.obs['guide_identity'].str.startswith('HCMV'), 'organism'] = 'Human Cytomegalovirus (HCMV)'

In [ ]:
adata.obs['perturbation_name'] = adata.obs['guide_identity'].str.split('_').str[-2]
adata.obs.loc[adata.obs['perturbation_name'] == '*', 'perturbation_name'] = 'non-targeting'

adata.obs['condition'] = adata.obs['perturbation_name']
adata.obs.loc[adata.obs['guide_identity'].str.startswith('control'), 'condition'] = 'control'
adata.obs.loc[adata.obs['guide_identity'] == '*', 'condition'] = 'non-targeting'
adata.obs['condition'] = adata.obs['condition'].apply(
    lambda x: 'targeting' if x not in ['control', 'non-targeting'] else x
)

In [ ]:
adata.obs

,guide_identity,read_count,UMI_count,coverage,gemgroup,good_coverage,number_of_cells,sample,cancer_type,cell_type,organism,perturbation_name,condition
cell_barcode,,,,,,,,,,,,,
AAACCTGAGAACTCGG-3,TRAPPC11_pMH2062,1332,86,15.488372,3,True,1,host,non-cancer,human foreskin fibroblasts (HFFs),Humans (Homo sapiens),TRAPPC11,targeting
AAACCTGAGAAGAAGC-4,STAT2_pMH2128,337,46,7.326087,4,True,1,host,non-cancer,human foreskin fibroblasts (HFFs),Humans (Homo sapiens),STAT2,targeting
AAACCTGAGAAGGCCT-2,STAT2_pMH2128,1162,72,16.138889,2,True,1,host,non-cancer,human foreskin fibroblasts (HFFs),Humans (Homo sapiens),STAT2,targeting
AAACCTGAGACACGAC-3,CASP9_pMH2178,1427,97,14.711340,3,True,1,host,non-cancer,human foreskin fibroblasts (HFFs),Humans (Homo sapiens),CASP9,targeting
AAACCTGAGAGAACAG-4,HCCS_pMH2186,397,48,8.270833,4,True,2,host,non-cancer,human foreskin fibroblasts (HFFs),Humans (Homo sapiens),HCCS,targeting
...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCATCTCGAGTA-2,HCMV_UL40_pMH2412,851,28,30.392857,2,True,1,host_and_virus,non-cancer,human foreskin fibroblasts (HFFs),Human Cytomegalovirus (HCMV),UL40,targeting
TTTGTCATCTCGCATC-4,HCMV_UL70_pMH2338,417,12,34.750000,4,True,1,host_and_virus,non-cancer,human foreskin fibroblasts (HFFs),Human Cytomegalovirus (HCMV),UL70,targeting
TTTGTCATCTGATTCT-3,human_HS6ST1_pMH2452,1677,64,26.203125,3,True,2,host_and_virus,non-cancer,human foreskin fibroblasts (HFFs),Humans (Homo sapiens),HS6ST1,targeting


In [7]:
adata.obs['crispr_type'] = adata.obs['sample'].map({
    'host': 'CRISPRi',
    'host_and_virus': 'CRISPRn'
})
adata.write_h5ad('GSE165291_merged.h5ad')